In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group3Data"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe(['Group3Data'])

In [0]:
aString = {}

kafkaList = []

for i in range(1000):
    try:
        msg = c.poll(timeout=15)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            print(aString)
            kafkaList.append(aString)
            c.commit(asynchronous=False)
    except Exception as e:
        print(e)

<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#9-BRONX', 'GraduationPercentageRate': '74.0', 'DropoutPercentageRate': '9.0', 'FundingPerStudent': '20785.08', 'TotalEnrolled': '29653.0', 'MalesEnrolled': '14989.0', 'FemalesEnrolled': '14664.0', 'Percentage of Minority Students': '98.7', 'HouseholdIncome': '47668.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c80fcb3c0>
{'DistrictName': 'NYCGEOGDIST#10-BRONX', 'GraduationPercentageRate': '80.0', 'DropoutPercentageRate': '7.0', 'FundingPerStudent': '18515.51', 'TotalEnrolled': '49381.0', 'MalesEnrolled': '25187.0', 'FemalesEnrolled': '24194.0', 'Percentage of Minority Students': '94.1', 'HouseholdIncome': '67737.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#11-BRONX', 'GraduationPercentageRate': '75.0', 'DropoutPercentageRate': '8.0', 'FundingPerStudent': '17704.26', 'TotalEnrolled': '35438.0', 'MalesEnrolled': '18643.0', 'FemalesEnrolled': '16795.0', 'Percentage of Minority Students': '91.1', 'HouseholdIncome': '75889.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c80fcb2c0>
{'DistrictName': 'NYCGEOGDIST#12-BRONX', 'GraduationPercentageRate': '63.0', 'DropoutPercentageRate': '12.0', 'FundingPerStudent': '20174.92', 'TotalEnrolled': '19427.0', 'MalesEnrolled': '10007.0', 'FemalesEnrolled': '9420.0', 'Percentage of Minority Students': '98.6', 'HouseholdIncome': '52783.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#13-BROOKLYN', 'GraduationPercentageRate': '90.0', 'DropoutPercentageRate': '2.0', 'FundingPerStudent': '16254.63', 'TotalEnrolled': '19994.0', 'MalesEnrolled': '10672.0', 'FemalesEnrolled': '9322.0', 'Percentage of Minority Students': '82.8', 'HouseholdIncome': '146314.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c80fcb3c0>
{'DistrictName': 'NYCGEOGDIST#14-BROOKLYN', 'GraduationPercentageRate': '82.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '19282.51', 'TotalEnrolled': '16583.0', 'MalesEnrolled': '8684.0', 'FemalesEnrolled': '7899.0', 'Percentage of Minority Students': '85.1', 'HouseholdIncome': '100410.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#15-BROOKLYN', 'GraduationPercentageRate': '71.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '18079.91', 'TotalEnrolled': '29846.0', 'MalesEnrolled': '14881.0', 'FemalesEnrolled': '14965.0', 'Percentage of Minority Students': '69.9', 'HouseholdIncome': '136417.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c80fcb3c0>
{'DistrictName': 'NYCGEOGDIST#16-BROOKLYN', 'GraduationPercentageRate': '60.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '24918.87', 'TotalEnrolled': '5425.0', 'MalesEnrolled': '2891.0', 'FemalesEnrolled': '2534.0', 'Percentage of Minority Students': '97.3', 'HouseholdIncome': '78197.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#17-BROOKLYN', 'GraduationPercentageRate': '74.0', 'DropoutPercentageRate': '6.0', 'FundingPerStudent': '19294.66', 'TotalEnrolled': '19019.0', 'MalesEnrolled': '9361.0', 'FemalesEnrolled': '9658.0', 'Percentage of Minority Students': '96.9', 'HouseholdIncome': '87778.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c80fcb2c0>
{'DistrictName': 'NYCGEOGDIST#18-BROOKLYN', 'GraduationPercentageRate': '67.0', 'DropoutPercentageRate': '6.0', 'FundingPerStudent': '20378.87', 'TotalEnrolled': '12417.0', 'MalesEnrolled': '6499.0', 'FemalesEnrolled': '5918.0', 'Percentage of Minority Students': '96.6', 'HouseholdIncome': '71335.0', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7f4c9efe75c0>
{'DistrictName': 'NYCGEOGDIST#19-BROOKLYN', 'GraduationPercentageRate': '79.0', 'DropoutPercentageRate': '6.0', 'FundingPerStudent': '20178.57', 'TotalEnrolled': '19389.0', 'MalesEnrolled': '10230.0', 'FemalesEnrolled': '9159.0', 'Percentage of Minority Students': '97.

In [0]:
print(kafkaList[16])
print(type(kafkaList[16]))

{'DistrictName': 'NYCGEOGDIST#25-QUEENS', 'GraduationPercentageRate': '76.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '16025.35', 'TotalEnrolled': '35583.0', 'MalesEnrolled': '18163.0', 'FemalesEnrolled': '17420.0', 'Percentage of Minority Students': '88.7', 'HouseholdIncome': '88695.0', 'FederallyFunded': 'Yes'}
<class 'dict'>

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame(kafkaList)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 10 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 DistrictName 719 non-null object
 1 GraduationPercentageRate 719 non-null object
 2 DropoutPercentageRate 719 non-null object
 3 FundingPerStudent 719 non-null object
 4 TotalEnrolled 719 non-null object
 5 MalesEnrolled 719 non-null object
 6 FemalesEnrolled 719 non-null object
 7 Percentage of Minority Students 719 non-null object
 8 HouseholdIncome 678 non-null object
 9 FederallyFunded 231 non-null object
dtypes: object(10)
memory usage: 56.3+ KB

In [0]:
finaldf = spark.createDataFrame(df)

In [0]:
finaldf.dtypes

Out[23]: [('DistrictName', 'string'),
 ('GraduationPercentageRate', 'string'),
 ('DropoutPercentageRate', 'string'),
 ('FundingPerStudent', 'string'),
 ('TotalEnrolled', 'string'),
 ('MalesEnrolled', 'string'),
 ('FemalesEnrolled', 'string'),
 ('Percentage of Minority Students', 'string'),
 ('HouseholdIncome', 'string'),
 ('FederallyFunded', 'string')]

In [0]:
from pyspark.sql.types import DecimalType, StringType, FloatType

output_df = finaldf \
  .withColumn("DistrictName", finaldf["DistrictName"].cast(StringType())) \
  .withColumn("GraduationPercentageRate", finaldf["GraduationPercentageRate"].cast(FloatType())) \
  .withColumn("DropoutPercentageRate", finaldf["DropoutPercentageRate"].cast(FloatType())) \
  .withColumn("FundingPerStudent", finaldf["FundingPerStudent"].cast(FloatType())) \
  .withColumn("TotalEnrolled", finaldf["TotalEnrolled"].cast(FloatType())) \
  .withColumn("MalesEnrolled", finaldf["MalesEnrolled"].cast(FloatType())) \
  .withColumn("FemalesEnrolled", finaldf["FemalesEnrolled"].cast(FloatType())) \
  .withColumn("Percentage of Minority Students", finaldf["Percentage of Minority Students"].cast(FloatType())) \
  .withColumn("HouseholdIncome", finaldf["HouseholdIncome"].cast(FloatType())) \
  .withColumn("FederallyFunded", finaldf["FederallyFunded"].cast(StringType())) 

In [0]:
output_df = output_df.withColumnRenamed("Percentage of Minority Students", "PercentageofMinorityStudents")

In [0]:
output_df.dtypes

Out[34]: [('DistrictName', 'string'),
 ('GraduationPercentageRate', 'float'),
 ('DropoutPercentageRate', 'float'),
 ('FundingPerStudent', 'float'),
 ('TotalEnrolled', 'float'),
 ('MalesEnrolled', 'float'),
 ('FemalesEnrolled', 'float'),
 ('PercentageofMinorityStudents', 'float'),
 ('HouseholdIncome', 'float'),
 ('FederallyFunded', 'string')]

In [0]:
database = "Capstone-Group3"
table = "dbo.nystatedistricts"
user = "stanleyperez"
password  = "Ezra!1223"
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

output_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()